# 1. package importation
This script use **pandas** as main functional package

In [6]:
import pandas as pd

In [15]:
admin1 = pd.read_csv('../Data/GHA/ADM1/admin.csv')
admin1_code_and_names = admin1[['3166-2 code','Subdivision name']]

## 2.2. Interventions

In [10]:
GHA_Intervention_data = pd.read_excel('../Data/GHA/Interventions/GHA_Intervention_data.xlsx','Data template ')
GHA_Intervention_data

,adm1,adm2,year,month,smc_yn,llins_yn,irs_yn,pbo_yn,smc1_num,smc2_num,smc3_num,smc4_num,llins_num,irs_cov,irs_popprot,irs_struct_target,irs_struct_spr
0,NaN,NaN,2008,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Northern,Gushegu,2008,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,89994,NaN,38874
2,Northern,Karaga,2008,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,75290,NaN,29114
3,Northern,Savelugu,2008,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,116579,NaN,44610
4,Northern,Tolon,2008,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,160043,NaN,68182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,Western North,Bodi,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,54740.0,NaN,NaN,NaN,NaN
747,Western North,Juaboso,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,96144.0,NaN,NaN,NaN,NaN
748,Western North,Sefwi Akontombra,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,61735.0,NaN,NaN,NaN,NaN
749,Western North,Sefwi-Wiawso,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,96608.0,NaN,NaN,NaN,NaN


In [16]:
pd.merge(admin1_code_and_names, GHA_Intervention_data, left_on='Subdivision name', right_on='adm1')

,3166-2 code,Subdivision name,adm1,adm2,year,month,smc_yn,llins_yn,irs_yn,pbo_yn,smc1_num,smc2_num,smc3_num,smc4_num,llins_num,irs_cov,irs_popprot,irs_struct_target,irs_struct_spr
0,GH-AF,Ahafo,Ahafo,Asunafo North Municipal,2015,NaN,0,1,0,0,NaN,NaN,NaN,NaN,72781.0,NaN,NaN,NaN,NaN
1,GH-AF,Ahafo,Ahafo,Asunafo South,2015,NaN,0,1,0,0,NaN,NaN,NaN,NaN,65327.0,NaN,NaN,NaN,NaN
2,GH-AF,Ahafo,Ahafo,Asutifi North,2015,NaN,0,1,0,0,NaN,NaN,NaN,NaN,42746.0,NaN,NaN,NaN,NaN
3,GH-AF,Ahafo,Ahafo,Asutifi South,2015,NaN,0,1,0,0,NaN,NaN,NaN,NaN,41169.0,NaN,NaN,NaN,NaN
4,GH-AF,Ahafo,Ahafo,Tano North Municipal,2015,NaN,0,1,0,0,NaN,NaN,NaN,NaN,53115.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,GH-WN,Western North,Western North,Bodi,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,54740.0,NaN,NaN,NaN,NaN
692,GH-WN,Western North,Western North,Juaboso,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,96144.0,NaN,NaN,NaN,NaN
693,GH-WN,Western North,Western North,Sefwi Akontombra,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,61735.0,NaN,NaN,NaN,NaN
694,GH-WN,Western North,Western North,Sefwi-Wiawso,2018,NaN,0,1,0,0,NaN,NaN,NaN,NaN,96608.0,NaN,NaN,NaN,NaN
